In [7]:
import pandas as pd
import contractions
import re
import nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import spacy

from sklearn.svm import SVC
from wordcloud import WordCloud
from nltk.util import ngrams
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from tqdm import tqdm
from PIL import Image
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical #Acomodar y_train y y_test pra poder entrenar el modelo
from tensorflow.keras.callbacks import EarlyStopping

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')
stopwords_en = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
tqdm.pandas()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jorge\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jorge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jorge\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\jorge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/1252/Procesamiento de Lenguaje Natural/Archivos/HateSpeech (1) (1).csv')
df = df[['class', 'tweet']]
df.head()

In [ ]:
"""
0: Hate Speech
1: Offensive:
2: Neither
"""

df['class'].unique()

In [ ]:
#La desproporción puede alterar el modelo
df['class'].value_counts().plot(kind='bar')

In [ ]:
undersampled_data= pd.DataFrame(columns=['class', 'tweet'])
undersampled_data = pd.concat([undersampled_data, df['class']], axis=0)
undersampled_data = pd.concat([undersampled_data, df['class']], axis=0)

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
def vectorize(texto):
  texto = texto.lower()
  texto = contractions.fix(texto)
  texto = re.sub(r'@[A-Za-z0-9]+','', texto)
  texto = re.sub(r'[^0-9A-Za-z \t]','', texto)
  texto = re.sub(r'\w+:\/\/\S+','', texto)
  texto = re.sub(r'\w+:\/\/\S+','', texto)
  texto = re.sub(r'rt','', texto)
  texto = word_tokenize(texto)
  texto = [token for token in texto if token not in stopwords_en]
  texto = [lemmatizer.lemmatize(word) for word in texto]
  texto = ' '.join(texto)
  texto = nlp(texto).vector
  return texto

In [ ]:
df['vector'] = df['tweet'].progress_apply(vectorize)
df.head()

In [ ]:
X = df['vector']
X = np.concatenate(X, axis=0).reshape(-1, 300)
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.7,
                                                    random_state=101,
                                                    stratify=y) #MANEJAR DESBALANCE ENTRE LA MULTICAPA



In [ ]:
y_train = to_categorical(y_train,
                         3) #Tenemos tres clases
y_test = to_categorical(y_test,
                         3) #Tenemos tres clases

In [ ]:
y_train

In [ ]:
model = Sequential() #Para agregar capa por capa
model.add(Dense(units = 128, activation = 'relu', input_dim = 300))
model.add(Dropout(0.305))#el 35 se va a apagar de manera aleatoria

model.add(Dense(units = 256, activation = 'relu'))
model.add(Dropout(0.305))
model.add(Dense(units = 3, activation = 'softmax')) #Usamos softmax al ser un problema no-binario
model.add(Dropout(0.305))#el 35 se va a apagar de manera aleatoria
model.compile(optimizer = Adam(learning_rate = 0.01), loss = 'categorical_crossentropy', metrics = ['accuracy'])
df.reset_index(implace=True)

In [ ]:
history = model.fit(X_train,
                    y_train,
                    batch_size = 10,
                    epochs = 10,
                    validation_data = (X_test, y_test),callbacks = [early_stopping])

In [ ]:
metrics = pd.DataFrame(history.history)
metrics

In [ ]:
sns.lineplot(data=metrics[['loss', 'val_loss']])

In [ ]:
sns.lineplot(data=metrics[['accuracy', 'val_accuracy']])

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
y_pred = np.argmax(y_pred, axis = -1)
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fat='2f')

In [ ]:
##cambios en la red neurona
